In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import google_api_key
gmaps.configure(api_key=google_api_key())

In [29]:
# STORE PART 1 RESULTS INTO A DATAFRAME

# Load the csv exported in Part I to a DataFrame
load_csv = "output_data/cities.csv"
cities_df = pd.read_csv(load_csv)
cities_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature,Cloud Cover,Humidity,Wind Speed,Date
0,0,Busselton,AU,-33.65,115.33,73.8,42,61,5.99,1607641825
1,1,Avarua,CK,-21.21,-159.78,84.2,5,74,20.80,1607641910
2,2,Puerto Ayora,EC,-0.74,-90.35,76.5,20,61,12.75,1607642044
3,3,Saldanha,ZA,-33.01,17.94,60.8,98,77,3.36,1607642045
4,4,Puerto Escondido,MX,15.85,-97.07,82.4,75,69,3.36,1607642047


In [59]:
# HUMIDITY HEATMAP

#Store latitude and longitude in places
places = cities_df[["Latitude", "Longitude"]]

#Store humidity in humidity
humidity = cities_df["Humidity"].astype(float)
humidity_max = humidity.max()

#Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(places, weights=humidity, dissipating=False, max_intensity=humidity_max, point_radius=2)

#Add layer
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
# CREATE NEW DATAFRAME FITTING WEATHER CRITERIA

# Narrow down the cities to fit weather conditions
# Drop any rows with null values..
narrowed_cities_df = cities_df.loc[(cities_df["Temperature"] > 70) & 
                                   (cities_df["Temperature"] < 80) & 
                                   (cities_df["Cloud Cover"] == 0),:]
narrowed_cities_df = narrowed_cities_df.dropna(how='any')
narrowed_cities_df.reset_index(inplace=True)
del narrowed_cities_df['index']
narrowed_cities_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature,Cloud Cover,Humidity,Wind Speed,Date
0,47,Molina,CL,-35.12,-71.28,75.20,0,33,8.05,1607641802
1,52,Keïta,NE,14.76,5.77,77.00,0,22,4.70,1607642105
2,85,Cabedelo,BR,-6.98,-34.83,78.80,0,78,9.17,1607642145
3,99,Cidreira,BR,-30.18,-50.21,70.75,0,81,15.73,1607642162
4,103,Phalaborwa,ZA,-23.94,31.14,78.80,0,47,4.23,1607642166


In [46]:
# HOTEL MAP

# Create a hotel_df
hotel_df = narrowed_cities_df.loc[:,["City","Country", 
                                     "Latitude", "Longitude"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
0,Molina,CL,-35.12,-71.28,
1,Keïta,NE,14.76,5.77,
2,Cabedelo,BR,-6.98,-34.83,
3,Cidreira,BR,-30.18,-50.21,
4,Phalaborwa,ZA,-23.94,31.14,


In [51]:
for index, row in hotel_df.iterrows():
    # Get city name, latitude, longitude from cities_df
    city_name = row["City"]
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    params = {"type": "hotel",
          "radius": 5000,
          "location": f"{latitude},{longitude}",
          "key": google_api_key()  
         }
    # Add keyword to params dict
    params["location"] = f"{latitude},{longitude}"

    # Form url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    # Get results
    results = response['results']
    #print(results)
    # Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.at[index,"Hotel Name"]= results[0]['name']
    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Skipping...")

    time.sleep(1.05)

# Print end of search once searching is completed
print("----------\nSearch Complete\n----------")

Retrieving Results for Index 0: Molina.
Closest hotel in Molina is Molina.
Retrieving Results for Index 1: Keïta.
Closest hotel in Keïta is Keita.
Retrieving Results for Index 2: Cabedelo.
Closest hotel in Cabedelo is Cabedelo.
Retrieving Results for Index 3: Cidreira.
Closest hotel in Cidreira is CIDREIRA.
Retrieving Results for Index 4: Phalaborwa.
Closest hotel in Phalaborwa is Phalaborwa.
Retrieving Results for Index 5: Giyani.
Closest hotel in Giyani is Giyani.
Retrieving Results for Index 6: Kidal.
Closest hotel in Kidal is Kidal.
Retrieving Results for Index 7: Makkah al Mukarramah.
Closest hotel in Makkah al Mukarramah is Mecca.
Retrieving Results for Index 8: Chabahar.
Closest hotel in Chabahar is Chabahar.
Retrieving Results for Index 9: Arlit.
Closest hotel in Arlit is Arlit.
Retrieving Results for Index 10: Rio Grande.
Closest hotel in Rio Grande is Rio Grande.
Retrieving Results for Index 11: Tonj.
Closest hotel in Tonj is Tonj.
Retrieving Results for Index 12: Muscat.
Clo

In [52]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Molina,CL,-35.12,-71.28,Molina
1,Keïta,NE,14.76,5.77,Keita
2,Cabedelo,BR,-6.98,-34.83,Cabedelo
3,Cidreira,BR,-30.18,-50.21,CIDREIRA
4,Phalaborwa,ZA,-23.94,31.14,Phalaborwa
5,Giyani,ZA,-23.30,30.72,Giyani
6,Kidal,ML,18.44,1.41,Kidal
7,Makkah al Mukarramah,SA,21.43,39.83,Mecca
8,Chabahar,IR,25.29,60.64,Chabahar
9,Arlit,NE,18.74,7.39,Arlit


In [54]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [58]:
# Add marker layer ontop of heat map & display figure
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [60]:
print(f"Humidity appears to be greater closer to the equator.")
print(f"As expected, the heatmap does not show high humidity in North America given the time of year.")
print(f"The marked hotels do not appear to be located at places with very high humidity.")

Humidity appears to be greater closer to the equator.
As expected, the heatmap does not show high humidity in North America given the time of year.
The marked hotels do not appear to be located at places with very high humidity.
